In [21]:

!pip install groq

from groq import Groq
import os


In [22]:

client = Groq(api_key="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")



From This cell, code for Task 1 start. Above are the api keys and installation of grok


In [23]:
class ConversationManager:
    def __init__(self, model="llama-3.1-8b-instant", k=3):
        self.model = model
        self.history = []
        self.summaries = []
        self.counter = 0
        self.k = k

    def add_message(self, role, content):
        """Add a user or assistant message"""
        self.history.append({"role": role, "content": content})
        self.counter += 1

    def get_history_limited(self, n=None, char_limit=None, word_limit=None):
        """Return truncated conversation history"""
        if n:  # limit by number of turns
            return self.history[-n:]
        elif char_limit:  # limit by characters
            text = ""
            truncated = []
            for msg in reversed(self.history):
                if len(text) + len(msg["content"]) > char_limit:
                    break
                truncated.insert(0, msg)
                text += msg["content"]
            return truncated
        elif word_limit:  # limit by words
            words = 0
            truncated = []
            for msg in reversed(self.history):
                msg_words = len(msg["content"].split())
                if words + msg_words > word_limit:
                    break
                truncated.insert(0, msg)
                words += msg_words
            return truncated
        else:
            return self.history

    def summarize_history(self):
        """Summarize the conversation using Groq API"""
        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "Summarize the following chat history briefly:"},
                {"role": "user", "content": str(self.history)}
            ]
        )
        summary = response.choices[0].message.content
        self.summaries.append(summary)
        # Replace history with summary for conciseness
        self.history = [{"role": "system", "content": f"Summary so far: {summary}"}]
        print("🔹 Conversation summarized!")
        return summary

    def maybe_summarize(self):
        """Summarize after every k-th message"""
        if self.counter % self.k == 0:
            return self.summarize_history()
        return None


In [24]:
# Create manager with summarization every 3rd message
manager = ConversationManager(model="llama-3.1-8b-instant", k=3)


sample_messages = [
    ("user", "Hi, I need help with Python loops."),
    ("assistant", "Sure! Python loops include for and while loops."),
    ("user", "Can you explain with a for loop example?"),
    ("assistant", "Yes. Example: for i in range(5): print(i)"),
    ("user", "Nice. What about while loops?"),
    ("assistant", "While loops repeat until a condition is false."),
    ("user", "Can I use break inside loops?"),
    ("assistant", "Yes, break exits the loop early.")
]

for role, content in sample_messages:
    manager.add_message(role, content)
    summary = manager.maybe_summarize()
    if summary:
        print("Summary after auto-trigger:", summary)

# Show truncated histories
print("\n--- Last 2 turns ---")
print(manager.get_history_limited(n=2))

print("\n--- Limit by 100 characters ---")
print(manager.get_history_limited(char_limit=100))

print("\n--- Limit by 30 words ---")
print(manager.get_history_limited(word_limit=30))

print("\n--- Stored summaries ---")
print(manager.summaries)


🔹 Conversation summarized!
Summary after auto-trigger: Here's a brief summary of the chat history:

A user requested help with Python loops and started with a basic inquiry. The assistant then explained that Python loops include for and while loops. The user then asked for an example of a for loop.
🔹 Conversation summarized!
Summary after auto-trigger: A user requested help with Python loops. The assistant explained that Python loops include for and while loops. Examples of each were provided, including a basic for loop and a description of how while loops work.

--- Last 2 turns ---
[{'role': 'user', 'content': 'Can I use break inside loops?'}, {'role': 'assistant', 'content': 'Yes, break exits the loop early.'}]

--- Limit by 100 characters ---
[{'role': 'user', 'content': 'Can I use break inside loops?'}, {'role': 'assistant', 'content': 'Yes, break exits the loop early.'}]

--- Limit by 30 words ---
[{'role': 'user', 'content': 'Can I use break inside loops?'}, {'role': 'assistant'

In [25]:
from groq import Groq
import json


In [26]:
client = Groq(api_key="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")


From here, code for task 2 starts


In [27]:
# Function schema definition for structured extraction
functions = [
    {
        "name": "extract_user_info",
        "description": "Extract personal details from chat conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "Full name of the user"},
                "email": {"type": "string", "description": "Email address"},
                "phone": {"type": "string", "description": "Phone number"},
                "location": {"type": "string", "description": "Location or city"},
                "age": {"type": "integer", "description": "Age in years"}
            },
            "required": ["name", "email", "phone", "location", "age"]
        }
    }
]


In [28]:
def extract_info_from_chat(chat_text, model="llama-3.1-8b-instant"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an information extraction assistant."},
            {"role": "user", "content": chat_text}
        ],
        functions=functions,
        function_call={"name": "extract_user_info"}
    )

    # Get structured function call output
    msg = response.choices[0].message
    if msg.function_call and msg.function_call.arguments:
      function_args = msg.function_call.arguments
    else:
      function_args = "{}"

    data = json.loads(function_args)


    return data


In [29]:
sample_chats = [
    "Hello, my name is John Doe. I live in New York. You can reach me at john@example.com or call 555-1234. I'm 29 years old.",
    "Hi, I'm Alice. Email: alice99@mail.com, Phone: 9876543210. I'm 34 years old and currently living in London.",
    "This is Raj from Bangalore, 41 years old. My contact is rajkumar@mail.in and phone is 9090909090."
]

for i, chat in enumerate(sample_chats, 1):
    extracted = extract_info_from_chat(chat)
    print(f"\n--- Chat {i} ---")
    print("Input:", chat)
    print("Extracted JSON:", extracted)



--- Chat 1 ---
Input: Hello, my name is John Doe. I live in New York. You can reach me at john@example.com or call 555-1234. I'm 29 years old.
Extracted JSON: {'age': 29, 'email': 'john@example.com', 'location': 'New York', 'name': 'John Doe', 'phone': '555-1234'}

--- Chat 2 ---
Input: Hi, I'm Alice. Email: alice99@mail.com, Phone: 9876543210. I'm 34 years old and currently living in London.
Extracted JSON: {'age': 34, 'email': 'alice99@mail.com', 'location': 'London', 'name': 'Alice', 'phone': '9876543210'}

--- Chat 3 ---
Input: This is Raj from Bangalore, 41 years old. My contact is rajkumar@mail.in and phone is 9090909090.
Extracted JSON: {'age': 41, 'email': 'rajkumar@mail.in', 'location': 'Bangalore', 'name': 'Raj', 'phone': '9090909090'}


In [30]:
from jsonschema import validate, ValidationError

# Define schema for validation (same as function parameters)
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

for i, chat in enumerate(sample_chats, 1):
    extracted = extract_info_from_chat(chat)
    print(f"\n--- Validating Chat {i} ---")
    try:
        validate(instance=extracted, schema=schema)
        print("✅ Valid output:", extracted)
    except ValidationError as e:
        print("❌ Validation failed:", e)



--- Validating Chat 1 ---
✅ Valid output: {'age': 29, 'email': 'john@example.com', 'location': 'New York', 'name': 'John Doe', 'phone': '555-1234'}

--- Validating Chat 2 ---
✅ Valid output: {'age': 34, 'email': 'alice99@mail.com', 'location': 'London', 'name': 'Alice', 'phone': '9876543210'}

--- Validating Chat 3 ---
✅ Valid output: {'age': 41, 'email': 'rajkumar@mail.in', 'location': 'Bangalore', 'name': 'Raj', 'phone': '9090909090'}
